In [1]:
import keras

Using TensorFlow backend.


In [2]:
!pip install np_utils

Data preprocessing

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
!pip install kaggle
!kaggle competitions download -c petfinder-adoption-prediction

/bin/sh: 1: kaggle: not found


In [3]:
data = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:13]
y = data.iloc[:, -1]

In [4]:
X.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import defaultdict

possDummyVals = defaultdict(set)

        

def addDummies(df, cols_for_dummies):
    for col in cols_for_dummies:

        df_col = pd.get_dummies(df[col], prefix = col, drop_first = True)
        possDummyVals[col] = set(df_col.columns)
        print('now possDummyVal for '+col+' : '+str(possDummyVals[col]))
        print('num of dummies: '+str(len(df_col.columns)))
        df = pd.concat([df, df_col], axis = 1)
        print("added dummy for "+col)
        df.drop([col], axis = 1, inplace = True)
        print('num of cols in df: '+str(len(df.columns)))
    return df

def preprocess(df):
    le = LabelEncoder()    
    df['Gender'] = le.fit_transform(df['Gender'])
    cols_for_dummies = ['Geography', 'NumOfProducts', 'Tenure']
    if len(df)==1:
        return encodeSingle(df, cols_for_dummies)
    return addDummies(df, cols_for_dummies)

In [6]:
def encodeSingle(df, cols_for_dummies):
    for col in cols_for_dummies:
        val_col = df[col][0]
        print(col+"_"+str(val_col))
        print('set '+ col+"_"+str(val_col) + 'to 1')
        print('num of cols: '+str(len(df.columns)))
        possible_vals = [val for val in possDummyVals[col] if val != val_col]
        print('other vals: '+str(possible_vals))
        for cat_val in possible_vals:
            if cat_val == str(col+"_"+str(val_col)):
                df[cat_val] = 1
            else:
                df[cat_val] = 0
        print('set other '+str(len(possible_vals)) + 'to 0')
        print('num of cols: '+str(len(df.columns)))
        df.drop([col], axis = 1, inplace = True)
        print('encoded: '+col+ ' now num of cols: '+str(len(df.columns)))
    return df

In [7]:
X = preprocess(X)

now possDummyVal for Geography : {'Geography_Spain', 'Geography_Germany'}
num of dummies: 2
added dummy for Geography
num of cols in df: 11
now possDummyVal for NumOfProducts : {'NumOfProducts_2', 'NumOfProducts_4', 'NumOfProducts_3'}
num of dummies: 3
added dummy for NumOfProducts
num of cols in df: 13
now possDummyVal for Tenure : {'Tenure_8', 'Tenure_7', 'Tenure_3', 'Tenure_2', 'Tenure_9', 'Tenure_6', 'Tenure_10', 'Tenure_4', 'Tenure_5', 'Tenure_1'}
num of dummies: 10
added dummy for Tenure
num of cols in df: 22


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [9]:
#feature scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
num_cols = ['CreditScore', 'Balance', 'EstimatedSalary']

In [10]:
X_train[num_cols] = sc.fit_transform(X_train[num_cols])
X_val[num_cols] = sc.transform(X_val[num_cols])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [11]:
y_train_temp = y_train.copy()
y_val_temp = y_val.copy()
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

Making the ANN

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [13]:
#Initializing the Keras model
classifier = Sequential()

In [14]:
#adding dense layers
classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu', input_dim = 22))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=22, units=12, kernel_initializer="uniform", activation="relu")`
  


In [15]:

classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, kernel_initializer="uniform", activation="relu")`
  


In [16]:
classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, kernel_initializer="uniform", activation="relu")`
  """Entry point for launching an IPython kernel.


In [17]:
classifier.add(Dense(output_dim = 2, init = 'uniform', activation = 'sigmoid'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2, kernel_initializer="uniform", activation="sigmoid")`
  """Entry point for launching an IPython kernel.


In [18]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1, mode='auto')]

In [19]:
#compile the ANN
from keras.optimizers import Adam
adam = Adam(lr = 1e-2)
classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
#training 
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight('balanced', [0, 1], y_train_temp)
classifier.fit(X_train, y_train, batch_size = 20, nb_epoch = 100, callbacks = callbacks, validation_split = 0.2, class_weight = class_weights)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 3s 395us/step - loss: 0.4676 - acc: 0.7909 - val_loss: 0.3828 - val_acc: 0.8063
Epoch 2/100
6400/6400 [==============================] - 2s 244us/step - loss: 0.4189 - acc: 0.8034 - val_loss: 0.3933 - val_acc: 0.8281
Epoch 3/100
6400/6400 [==============================] - 2s 245us/step - loss: 0.4115 - acc: 0.8189 - val_loss: 0.3834 - val_acc: 0.8300
Epoch 4/100
6400/6400 [==============================] - 2s 251us/step - loss: 0.4075 - acc: 0.8215 - val_loss: 0.3628 - val_acc: 0.8438
Epoch 5/100
6400/6400 [==============================] - 2s 255us/step - loss: 0.3992 - acc: 0.8295 - val_loss: 0.3684 - val_acc: 0.8350
Epoch 6/100
6400/6400 [==============================] - 2s 237us/step - loss: 0.3981 - acc: 0.8266 - val_loss: 0.3639 - val_acc: 0.8419
Epoch 7/100
6400/6400 [==============================] - 2s 241us/step - loss: 0.3969 - acc: 0.8351 - val_loss: 0.3586 - val_acc: 

Now that model has been trained to 85% accuracy , start with test/validation set

In [21]:
classifier.evaluate(X_val, y_val)

2000/2000 [==============================] - 0s 59us/step


[0.37075501108169556, 0.84475]

In [22]:
y_pred = classifier.predict_classes(X_val)

In [23]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val_temp, y_pred)

In [24]:
cm

array([[1578,   29],
       [ 281,  112]])

In [27]:
(1578+112)/(2000)

0.845

In [26]:
y_pred_prob = classifier.predict(X_val)
y_pred1 = (y_pred_prob[:, 1] > 0.5)
cm1 = confusion_matrix(y_val_temp, y_pred1)
cm1

#IMP: CAN SET CONFIDENCE THIS WAY

array([[1578,   29],
       [ 282,  111]])

In [ ]:
#same results

In [26]:
(1543+164)/(1543+164+235+58) #accuracy is calculated using this 0.5 confidence method

0.8535

In [28]:
rows_list = []
row_dict = {'Geography': 'France', 'CreditScore': 600, 'Gender': 'Male', 'Age': 40, 'Tenure': 3, 'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 50000}
rows_list.append(row_dict)
test_df = pd.DataFrame(rows_list)


In [29]:
test_df.columns

Index(['Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'Gender',
       'Geography', 'HasCrCard', 'IsActiveMember', 'NumOfProducts', 'Tenure'],
      dtype='object')

In [30]:
test_df = preprocess(test_df)
test_df[num_cols] = sc.transform(test_df[num_cols])

Geography_France
set Geography_Franceto 1
num of cols: 10
other vals: ['Geography_Spain', 'Geography_Germany']
set other 2to 0
num of cols: 12
encoded: Geography now num of cols: 11
NumOfProducts_2
set NumOfProducts_2to 1
num of cols: 11
other vals: ['NumOfProducts_2', 'NumOfProducts_4', 'NumOfProducts_3']
set other 3to 0
num of cols: 14
encoded: NumOfProducts now num of cols: 13
Tenure_3
set Tenure_3to 1
num of cols: 13
other vals: ['Tenure_8', 'Tenure_7', 'Tenure_3', 'Tenure_2', 'Tenure_9', 'Tenure_6', 'Tenure_10', 'Tenure_4', 'Tenure_5', 'Tenure_1']
set other 10to 0
num of cols: 23
encoded: Tenure now num of cols: 22


In [31]:
col_val = [c for c in test_df.columns if c not in X.columns]
col_val

[]

In [32]:
pred_class = classifier.predict_classes(test_df)

In [33]:
pred_class

array([0])